<a href="https://colab.research.google.com/github/poojamahajan0712/AI_ML_concepts/blob/main/Quantization/Quantization_NB2_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reference - https://medium.com/game-of-bits/optimizing-tensorflow-models-using-quantization-fb4d09b46fac
# https://ai.google.dev/edge/litert/models/post_training_quantization

* key idea behind quantization - These techniques aim at providing smaller and faster models while keeping the performance of the models almost similar.
* Post-training quantization -  the deep learning model is trained with FP-32 tensors and later converted to INT-8(or float-16) in order to get a smaller and faster model for deployment. it is a bit more stable than quantization aware training and easy to use.
* In post-quantization techniques, we train the deep learning model normally and save the weights. These weights are later converted into TFLite format and quantized.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.models import Model


%matplotlib inline

In [2]:
#loading dataset
digits = load_digits()
images = digits['images']
labels = digits['target']
print (images.shape, labels.shape)

#Splitting Data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Encoding Labels
def get_encoded_labels(target):
    output=np.zeros((len(target),10))
    for ix, value in enumerate(target):
        output[ix][target[ix]] = 1
    return output

Y_train = get_encoded_labels(y_train)
Y_test = get_encoded_labels(y_test)
print (Y_train.shape, Y_test.shape)


(1797, 8, 8) (1797,)
(1347, 8, 8, 1) (450, 8, 8, 1) (1347,) (450,)
(1347, 10) (450, 10)


In [3]:


input_layer = Input(shape=(8, 8, 1))
layer = Conv2D(64, (3,3), activation='relu')(input_layer)
layer = Conv2D(32, (3,3), activation='relu')(layer)
layer = Conv2D(32, (3,3), activation='relu')(layer)
layer = Flatten()(layer)
features = Dense(32, activation='relu')(layer)
output = Dense(10, activation='softmax')(features)


model = Model(inputs=input_layer, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8, 8, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 6, 6, 64)            │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 4, 4, 32)            │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 2, 2, 32)            │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,810 (128.16 KB)

 Trainable params: 32,810 (128.16 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.3816 - loss: 1.7980 - val_accuracy: 0.8844 - val_loss: 0.3852
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8820 - loss: 0.3848 - val_accuracy: 0.9467 - val_loss: 0.1853
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9243 - loss: 0.2093 - val_accuracy: 0.9400 - val_loss: 0.2012
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.9455 - loss: 0.1583 - val_accuracy: 0.9733 - val_loss: 0.1180
Epoch 5/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9669 - loss: 0.1053 - val_accuracy: 0.9644 - val_loss: 0.1074
Epoch 6/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9575 - loss: 0.1213 - val_accuracy: 0.9778 - val_loss: 0.0717
Epoch 7/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9910 - loss: 0.0438 - val_accuracy: 0.9844 - val_loss: 0.0651
Epoch 8/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9950 - loss: 0.0238 - val_accuracy: 0.9844 - v

In [5]:
def get_test_accuracy(predictions, target):
    correct = 0
    for ix, pred in enumerate(predictions):
        true_value = target[ix]
        if pred[true_value] == max(pred):
            correct += 1
    return correct*100/len(target)
predictions = model.predict(X_test)
get_test_accuracy(predictions, y_test)

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


98.66666666666667

In [22]:
model.save("saved_model.keras")

### Post training quantization

1. Dynamic range quantization -
* Dynamic range quantization provides reduced memory usage and faster computation without you having to provide a representative dataset for calibration.
- The key point here is that you don't need a representative dataset for calibration- This means you can apply dynamic range quantization directly to an already-trained model without needing additional data to fine-tune the quantization process
* This type of quantization, statically quantizes only the weights from floating point to integer at conversion time, which provides 8-bits of precision
* To further reduce latency during inference, "dynamic-range" operators dynamically quantize activations based on their range to 8-bits and perform computations with 8-bit weights and activations.

In [6]:
## converting to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

## applying quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

Saved artifact at '/tmp/tmps7opvf22'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8, 8, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132638200706880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132640508996672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201247728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201237696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201285264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201283152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201284208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201293712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201294240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638201296528: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [10]:
with open('quantized_model.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [19]:

interpreter = tf.lite.Interpreter(model_path="quantized_model.tflite")

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()

print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([1, 8, 8, 1], dtype=int32), 'shape_signature': array([-1,  8,  8,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 18, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [27]:
input_details[0]

{'name': 'serving_default_keras_tensor:0',
 'index': 0,
 'shape': array([1, 8, 8, 1], dtype=int32),
 'shape_signature': array([-1,  8,  8,  1], dtype=int32),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [30]:
# interpreter.set_tensor() is used to set or assign the input data (the image) to the input tensor for the model to process.
# After calling set_tensor() to assign the input, you need to invoke the model to run the inference and calculate the outputs.
# get_tensor(): This method retrieves the output tensor after the inference is complete.After the model has run the inference, the results are stored in the output tensor, and get_tensor() allows you to access these results.


predictions = []
for img in X_test:
    interpreter.set_tensor(input_details[0]['index'], [img.astype('float32')])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(output_data[0])

predictions = np.array(predictions)
get_test_accuracy(predictions, y_test)

98.66666666666667

comparing model size

In [38]:
def get_model_size(model_path):
  model_size = os.path.getsize(model_path)
  model_size_mb = model_size / (1024*1024)
  print(f"model size: {model_size_mb:.2f} MB")

In [39]:
model_path = "/content/saved_model.keras"
get_model_size(model_path)

model size: 0.41 MB


In [40]:
model_path = "/content/quantized_model.tflite"
get_model_size(model_path)

model size: 0.04 MB


In [21]:


# Path to the quantized model
model_path = 'quantized_model.tflite'
# Get the size in bytes
model_size = os.path.getsize(model_path)
# Convert to kilobytes (KB) or megabytes (MB)
model_size_mb = model_size_kb / 1024
print(f"Quantized model size: {model_size_mb:.2f} MB")


Quantized model size: 39.63 KB
Quantized model size: 0.04 MB
